# 1️⃣ Training an Adapter for a Transformer model

In this notebook, we train an adapter for a **RoBERTa** ([Liu et al., 2019](https://arxiv.org/pdf/1907.11692.pdf)) model for sequence classification on a **sentiment analysis** task using [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers), the _AdapterHub_ adaptation of HuggingFace's _transformers_ library.

If you're unfamiliar with the theoretical parts of adapters or the AdapterHub framework, check out our [introductory blog post](https://adapterhub.ml/blog/2020/11/adapting-transformers-with-adapterhub/) first.

We train a **Task Adapter** for a pre-trained model here. Most of the code is identical to a full finetuning setup using HuggingFace's transformers. For comparison, have a look at the [same guide using full finetuning](https://colab.research.google.com/drive/1brXJg5Mokm8h3shxqPRnoIsRwHQoncus?usp=sharing).

For training, we use the [movie review dataset by Pang and Lee (2005)](http://www.cs.cornell.edu/people/pabo/movie-review-data/). It contains movie reviews  from Rotten Tomatoes which are either classified as positive or negative. We download the dataset via HuggingFace's [datasets](https://github.com/huggingface/datasets) library.

## Installation

First, let's install the required libraries:

In [1]:
!pip install -U adapter-transformers
!pip install datasets

     |████████████████████████████████| 2.9 MB 7.2 MB/s 
     |████████████████████████████████| 3.3 MB 60.0 MB/s 
     |████████████████████████████████| 61 kB 407 kB/s 
     |████████████████████████████████| 895 kB 51.3 MB/s 
     |████████████████████████████████| 596 kB 46.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 298 kB 6.6 MB/s 
     |████████████████████████████████| 1.1 MB 57.1 MB/s 
     |████████████████████████████████| 132 kB 63.7 MB/s 
     |████████████████████████████████| 243 kB 58.1 MB/s 
     |████████████████████████████████| 160 kB 83.8 MB/s 
     |████████████████████████████████| 192 kB 73.7 MB/s 
     |████████████████████████████████| 271 kB 71.0 MB/s 


## Dataset Preprocessing

Before we start to train our adapter, we first prepare the training data. Our training dataset can be loaded via HuggingFace `datasets` using one line of code:

In [2]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("yxchar/citation_intent-tlm")
dataset.num_rows

Using custom data configuration yxchar___citation_intent-tlm-1b778e179b05f85c


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/400k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/32.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.4k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/yxchar___citation_intent-tlm-1b778e179b05f85c/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'test': 139, 'train': 1688, 'validation': 114}

Every dataset sample has an input text and a binary label:

In [3]:
dataset['train']['label'][0]

0

Now, we need to encode all dataset samples to valid inputs for our Transformer model. Since we want to train on `roberta-base`, we load the corresponding `RobertaTokenizer`. Using `dataset.map()`, we can pass the full dataset through the tokenizer in batches:

In [4]:
import numpy as np
nLabels = len(np.unique(dataset['train']['label']))
nLabels

6

In [5]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=512, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset.rename_column_("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: rename_column_ is deprecated and will be removed in the next major version of datasets. Use DatasetDict.rename_column instead.
  if sys.path[0] == '':


Now we're ready to train our model...

## Training

We use a pre-trained RoBERTa model from HuggingFace. We use `RobertaModelWithHeads`, a class unique to `adapter-transformers`, which allows us to add and configure prediction heads in a flexibler way.

In [6]:
dataset['train']['labels']

tensor([0, 1, 0,  ..., 0, 2, 1])

In [7]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=nLabels,
)
model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

**Here comes the important part!**

We add a new adapter to our model by calling `add_adapter()`. We pass a name (`"rotten_tomatoes"`) and [the type of adapter](https://docs.adapterhub.ml/adapters.html#adapter-types) (task adapter). Next, we add a binary classification head. It's convenient to give the prediction head the same name as the adapter. This allows us to activate both together in the next step. The `train_adapter()` method does two things:

1. It freezes all weights of the pre-trained model so only the adapter weights are updated during training.
2. It activates the adapter and the prediction head such that both are used in every forward pass.

In [8]:
adaptername = 'CitationIntent'

# Add a new adapter
model.add_adapter(adaptername)
# Add a matching classification head
model.add_classification_head(
    adaptername,
    num_labels=nLabels
    # id2label={ 0: "👎", 1: "👍"}
  )
# Activate the adapter
model.train_adapter(adaptername)

For training, we make use of the `Trainer` class built-in into `transformers`. We configure the training process using a `TrainingArguments` object and define a method that will calculate the evaluation accuracy in the end. We pass both, together with the training and validation split of our dataset, to the trainer instance.

**Note the differences in hyperparameters compared to full finetuning.** Adapter training usually required a few more training epochs than full finetuning.

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'attention_mask', 'id', 'input_ids', 'labels', 'text'],
        num_rows: 1688
    })
    test: Dataset({
        features: ['Unnamed: 0', 'attention_mask', 'id', 'input_ids', 'labels', 'text'],
        num_rows: 139
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'attention_mask', 'id', 'input_ids', 'labels', 'text'],
        num_rows: 114
    })
})

In [10]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [11]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

Start the training 🚀

In [12]:
trainer.train()

***** Running training *****
  Num examples = 1688
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2120


Step,Training Loss
200,1.310500
400,1.009600
600,0.858400
800,0.712300
1000,0.595300
1200,0.522300
1400,0.453900
1600,0.419400
1800,0.368700
2000,0.345800


Saving model checkpoint to ./training_output/checkpoint-500
Configuration saved in ./training_output/checkpoint-500/CitationIntent/adapter_config.json
Module weights saved in ./training_output/checkpoint-500/CitationIntent/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-500/CitationIntent/head_config.json
Module weights saved in ./training_output/checkpoint-500/CitationIntent/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-500/CitationIntent/head_config.json
Module weights saved in ./training_output/checkpoint-500/CitationIntent/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-500/CitationIntent/head_config.json
Module weights saved in ./training_output/checkpoint-500/CitationIntent/pytorch_model_head.bin
Saving model checkpoint to ./training_output/checkpoint-1000
Configuration saved in ./training_output/checkpoint-1000/CitationIntent/adapter_config.json
Module weights saved in ./training_output/checkpoint-10

TrainOutput(global_step=2120, training_loss=0.640985479894674, metrics={'train_runtime': 1353.3736, 'train_samples_per_second': 24.945, 'train_steps_per_second': 1.566, 'total_flos': 9036971107614720.0, 'train_loss': 0.640985479894674, 'epoch': 20.0})

Looks good! Let's evaluate our adapter on the validation split of the dataset to see how well it learned:

In [13]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 114
  Batch size = 16


{'epoch': 20.0,
 'eval_acc': 0.7280701754385965,
 'eval_loss': 0.8673890829086304,
 'eval_runtime': 2.2915,
 'eval_samples_per_second': 49.75,
 'eval_steps_per_second': 3.491}

We can put our trained model into a `transformers` pipeline to be able to make new predictions conveniently:

At last, we can also extract the adapter from our model and separately save it for later reuse. Note the size difference compared to a full model!

In [14]:
import torch
device='cuda'

test_dataset = dataset['test']
bsz = 16
# bsz = test_dataset.num_rows
i = 0
batches = []
while i<test_dataset.num_rows:
  batches.append(test_dataset[i:i+bsz])
  i+=bsz


# Put model in evaluation mode
model.to(device)
model.eval()


# Tracking variables for storing ground truth and predictions 
predictions , true_labels = [], []

# Prediction Loop
for i,batch in enumerate(batches):
  print(i)

 
 
  # Unpack the inputs from our dataloader and move to GPU/accelerator 
 
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)
  labels = batch['labels'].to(device)

  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(input_ids, attention_mask=attention_mask, 
                         labels=labels)

  logits = outputs[1]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

0
1
2
3
4
5
6
7
8


In [15]:
labels = np.concatenate(true_labels)
prds = np.concatenate(predictions).argmax(axis=1)
from sklearn.metrics import f1_score, classification_report
f1_score(labels, prds, average='micro')

0.7266187050359713

In [17]:
print(classification_report(labels, prds, labels=list(range(nLabels)), digits=4))

              precision    recall  f1-score   support

           0     0.8133    0.8592    0.8356        71
           1     0.6923    0.6923    0.6923        26
           2     0.7000    0.5600    0.6222        25
           3     0.5000    0.6000    0.5455         5
           4     0.2500    0.2857    0.2667         7
           5     0.7500    0.6000    0.6667         5

    accuracy                         0.7266       139
   macro avg     0.6176    0.5995    0.6048       139
weighted avg     0.7284    0.7266    0.7253       139



In [18]:
adaptername = 'CitationIntent'

In [19]:
model.save_adapter("./final_adapter", adaptername)
!ls -lh final_adapter

Configuration saved in ./final_adapter/adapter_config.json
Module weights saved in ./final_adapter/pytorch_adapter.bin
Configuration saved in ./final_adapter/head_config.json
Module weights saved in ./final_adapter/pytorch_model_head.bin


total 5.8M
-rw-r--r-- 1 root root  580 Dec 15 05:20 adapter_config.json
-rw-r--r-- 1 root root  466 Dec 15 05:20 head_config.json
-rw-r--r-- 1 root root 3.5M Dec 15 05:20 pytorch_adapter.bin
-rw-r--r-- 1 root root 2.3M Dec 15 05:20 pytorch_model_head.bin


In [ ]:
from google.colab import files
!zip -r /content/file.zip /content
files.download("/content/file.zip")

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2021.12.03/ (stored 0%)
  adding: content/.config/logs/2021.12.03/14.32.50.522723.log (deflated 53%)
  adding: content/.config/logs/2021.12.03/14.33.16.964195.log (deflated 54%)
  adding: content/.config/logs/2021.12.03/14.33.37.701606.log (deflated 53%)
  adding: content/.config/logs/2021.12.03/14.33.36.903459.log (deflated 54%)
  adding: content/.config/logs/2021.12.03/14.33.09.955489.log (deflated 86%)
  adding: content/.config/logs/2021.12.03/14.32.30.027140.log (deflated 91%)
  adding: content/.config/.last_update_check.json (deflated 24%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.con

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Share your work!**

The next step after training is to share our adapter with the world via _AdapterHub_. [Read our guide](https://docs.adapterhub.ml/contributing.html) on how to prepare the adapter module we just saved and contribute it to the Hub!

➡️ Also continue with [the next Colab notebook](https://colab.research.google.com/github/Adapter-Hub/adapter-transformers/blob/master/notebooks/02_Adapter_Inference.ipynb) to learn how to use adapters from the Hub.